In [1]:
import torch
math_acts = torch.load('../../res/math_pythia_70m_deduped_res_sm_freq_mean_global.pt')
acts = torch.load('../../res/pythia_70m_deduped_res_sm_freq_mean_global.pt')

/tmp/ipykernel_5322/3362653199.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  math_acts = torch.load('../../res/math_pythia_70m_deduped_res_sm_freq_mean_global.pt')
/tm

In [2]:
act_0 = acts[0]
act_0.shape

torch.Size([32768])

In [6]:
sorted_act_0, indices = act_0.sort()
indices

tensor([    3,     6,     9,  ..., 12877, 29435, 32375], device='cuda:0')

In [9]:
d_sae = 32576
num_act0 = sorted_act_0 * d_sae
num_act0

tensor([   0.0000,    0.0000,    0.0000,  ..., 1508.2632, 1508.5260,
        1598.4976], device='cuda:0')

In [12]:
math_act0 = math_acts[0]

In [14]:
indices

tensor([    3,     6,     9,  ..., 12877, 29435, 32375], device='cuda:0')

In [20]:
arranged_math_act0 = math_act0[indices]
arranged_math_act0.nonzero().shape

torch.Size([437, 1])

In [16]:
import plotly.express as px
fig = px.line(sorted_act_0.cpu().numpy(), title='Activation values of the first layer of the Pythia model')

fig.add_scatter(y=arranged_math_act0.cpu().numpy(), mode='lines', name='Math Activation')
fig.show()

In [23]:
acts[0].mean()
math_act0.mean()

tensor(0.0001, device='cuda:0')

In [24]:
import datasets
ds = datasets.load_dataset("iamtarun/code_contest_python3_alpaca")


/mnt/c/Users/huohu/Documents/code/SAEGeometry/config/saegeometry-1tp4usyN-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Generating test split: 100%|██████████| 122/122 [00:00<00:00, 4260.45 examples/s]


In [28]:
ds['train'][0]['code']

"from collections import deque\n\ndef solve(adj, m, k, uv):\n    n = len(adj)\n    nn = [len(a) for a in adj]\n    q = deque()\n    for i in range(n):\n        if nn[i] < k:\n            q.append(i)\n    while q:\n        v = q.popleft()\n        for u in adj[v]:\n            nn[u] -= 1\n            if nn[u] == k-1:\n                q.append(u)\n    res = [0]*m\n    nk = len([1 for i in nn if i >= k])\n    res[-1] = nk\n    for i in range(m-1, 0, -1):\n        u1, v1 = uv[i]\n\n        if nn[u1] < k or nn[v1] < k:\n            res[i - 1] = nk\n            continue\n        if nn[u1] == k:\n            q.append(u1)\n            nn[u1] -= 1\n        if not q and nn[v1] == k:\n            q.append(v1)\n            nn[v1] -= 1\n\n        if not q:\n            nn[u1] -= 1\n            nn[v1] -= 1\n            adj[u1].remove(v1)\n            adj[v1].remove(u1)\n\n        while q:\n            v = q.popleft()\n            nk -= 1\n            for u in adj[v]:\n                nn[u] -= 1\n   

In [30]:
import sae_lens
layers = 6
device = "cuda" if torch.cuda.is_available() else "cpu"
sae_list = []
release = "pythia-70m-deduped-res-sm"
sae_id = f"blocks.0.hook_resid_pre"
model_name = "pythia-70m-deduped"
sae_list.append(
    sae_lens.SAE.from_pretrained(
        release=release, sae_id=sae_id, device=device
    )[0]
)
for layer in range(layers):
    sae_id = f"blocks.{layer}.hook_resid_post"
    sae_list.append(
        sae_lens.SAE.from_pretrained(
            release=release, sae_id=sae_id, device=device
        )[0]
    )

model = sae_lens.HookedSAETransformer.from_pretrained(model_name).to(device)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loaded pretrained model pythia-70m-deduped into HookedTransformer
Moving model to device:  cuda


In [32]:
from tqdm import tqdm
save_name="math_gemme_freq_mean_global.pt"
dataset = ds['train']
doc_len = 0
freq_mean_global = 0
layers = 6
freqs = torch.zeros(layers + 1, sae_list[0].cfg.d_sae).to(device)
for idx in tqdm(range(len(dataset))):
    # loop begin, fuck indent
    example = dataset[idx]
    tokens = model.to_tokens([example["code"]], prepend_bos=True)
    _, cache = model.run_with_cache_with_saes(tokens, saes=sae_list)
    local_doc_len = cache["blocks.0.hook_resid_post.hook_sae_acts_post"].shape[1]
    freq = torch.zeros_like(freqs)
    for layer in range(layers):
        prompt = f"blocks.{layer}.hook_resid_pre.hook_sae_acts_post"
        prompt2 = f"blocks.{layer}.hook_resid_post.hook_sae_acts_post"
        if layer == 0:
            freq[layer] = (cache[prompt] > 1e-3)[0].sum(0) / local_doc_len

        freq[layer + 1] = (cache[prompt2] > 1e-3)[0].sum(0) / local_doc_len
    new_doc_len = doc_len + local_doc_len
    if idx == 0:
        freq_mean_global = freq
    else:
        freq_mean_global = (
            freq_mean_global * doc_len / new_doc_len
            + freq * local_doc_len / new_doc_len
        )
    doc_len = new_doc_len

  3%|▎         | 210/8139 [00:05<03:13, 40.94it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f4267633860>>
Traceback (most recent call last):
  File "/mnt/c/Users/huohu/Documents/code/SAEGeometry/config/saegeometry-1tp4usyN-py3.12/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
  9%|▉         | 772/8139 [00:18<02:55, 42.08it/s]


KeyboardInterrupt: 

In [36]:
sae_list[0].W_dec.shape

torch.Size([32768, 512])

In [38]:
sae_list[0].b_dec

Parameter containing:
tensor([-1.8979e-03,  1.6620e-03,  4.0893e-03, -7.1394e-03,  9.2102e-04,
        -6.1163e-03,  1.7887e-04, -1.2226e-03, -5.4798e-02, -9.9980e-04,
         1.6866e-03,  6.1169e-03, -6.3277e-03,  1.0365e-03,  6.0435e-03,
         2.0783e-03, -1.8737e-03, -3.9198e-03, -4.4370e-05, -1.5410e-03,
         3.2355e-04, -4.6535e-03,  2.5255e-03,  5.9797e-03,  6.4882e-03,
         4.3351e-03,  3.3857e-03,  5.4833e-03, -1.9323e-03, -3.8319e-03,
        -3.5918e-03, -5.6028e-04,  5.5307e-04,  2.4007e-03,  1.0272e-02,
         2.3721e-03,  1.3615e-04,  8.6675e-03, -2.0252e-04, -2.1441e-03,
         2.0148e-03,  1.4181e-03, -2.0505e-03, -5.1920e-03, -3.3693e-03,
         3.9936e-03,  1.0863e-02,  3.0803e-03, -3.5896e-03, -8.5715e-04,
         3.5915e-03, -9.4086e-04, -2.4216e-03,  8.6369e-04, -4.4462e-03,
        -4.5297e-03, -4.1144e-03, -1.5966e-02,  4.2468e-03,  8.1251e-03,
        -2.4952e-03, -1.2161e-03, -4.8262e-03,  3.2336e-03, -4.1606e-03,
         1.6066e-03,  4.0910e